In [39]:
import numpy as np
import sympy
import scipy.optimize as opt
import copy
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from scipy.sparse import spdiags

In [40]:
# Parameters
rho = 0.035
r = 0.05
delta = 0.01
c_param = 2
l_param = 2
B_param = 2

params = [rho, r, delta, c_param, l_param]

T = 70

In [41]:
# Define utility functions
def u(c, c_param):
    return (c**(1-c_param))/(1-c_param)

def u_prime(c, c_param):
    return c**(-c_param)

def u_prime_inverse(x, c_param):
    return x**(-1/c_param)

def uh(h, l_param):
    return ((1-h)**(1-l_param))/(1-l_param)

def uh_prime(h, l_param):
    if h >= 1:
        return -1e99

    return -(1-h)**(-l_param)

def uh_prime_inverse(x, l_param):
    return 1 - ((-x)**(-1/l_param))

In [42]:
# Labor market functions
def g(x):
    return 1.25 - (x*(1.25**0.5 - 0.5) + 0.5)**2

def g_prime(x):
    return -2*(x*(1.25**0.5 - 0.5) + 0.5)*(1.25**0.5 - 0.5)

def g_prime_inverse(x):
    return (x/(-2*(1.25**0.5 - 0.5)) - 0.5)/(1.25**0.5 - 0.5)

Create grids

In [43]:
# State grids
na = 20 # number of asset grid points
nk = 20 # number of human capital grid points

amin = 1
amax = 10

kmin = 2
kmax = 10

a_vect = amin + np.linspace(0, 1, na)*(amax - amin) # Vector of A
da = (amax - amin)/(na - 1) # delta in A
aa = np.repeat(a_vect[:, None], nk, axis = 1) # Grid of A

k_vect = kmin + np.linspace(0, 1, nk)*(kmax - kmin) # Vector of A
dk = (kmax - kmin)/(nk - 1) # delta in A
kk = np.repeat(k_vect[None, :], na, axis = 0) # Grid of K

In [44]:
# Time parameters
T = 3
dt = 0.25

In [45]:
# Preallocate fwd/bwd derivatives, value function grid
dVaF = np.zeros((na, nk))
dVaB = np.zeros((na, nk))
dVkF = np.zeros((na, nk))
dVkB = np.zeros((na, nk))

v = np.zeros((na, nk, T))

In [46]:
# Terminal value
small_number1 = 1
small_number2 = 1
v_terminal = small_number1*((small_number2 + aa)**(1-B_param))/(1-B_param)

Compute steady state values

In [47]:
# Steady state requires solving FOC + (drift = 0) = 5 equations, with 5 variables (c, h, k, dVa, dVk)
i = 0
j = 0
dVa0 = np.zeros((na, nk))
dVk0 = np.zeros((na, nk))
C0 = np.zeros((na, nk))
H0 = np.zeros((na, nk))
X0 = np.zeros((na, nk))

In [48]:
# This function returns errors in the FOCs and drifts
def steady_state_err(guess, a, k, param):
    rho, r, delta, c_param, l_param = params
    dVa, dVk, c, h, x = guess

    mu_a = h*k*g(x) + r*a - c
    mu_k = h*k*x - delta*k
    focc = u_prime(c, c_param) - dVa
    foch = uh_prime(h, l_param) + dVa*k*g(x) + dVk
    focx = dVa*h*k*g_prime(x) + dVk*h*k

    return mu_a, mu_k, focc, foch, focx

In [49]:
# Loop over all grid points
for i in range(na):
    for j in range(nk):
        dVa0[i, j], dVk0[i, j], C0[i, j], H0[i, j], X0[i, j] = opt.fsolve(steady_state_err, [0, 0, 0.1, 0.5, 0.5], args = (aa[i, j], kk[i, j], params))

In [50]:
# Function computes drift in a given a dVa
def boundary_dVa(dVa, dVkj, a, kj, l_param):
    xstar = np.clip(g_prime_inverse(-dVkj/dVa), 0, 1)
    hstar = np.clip(uh_prime_inverse(-dVa*kj*g(xstar) - dVkj, l_param), 0, 1)
    return hstar*kj*g(xstar) + r*a - dVa**(-1/c_param)

In [51]:
# Function computes drift in k for a given dVk
def boundary_dVk(dVk, dVai, ai, k, l_param):
        xstar = np.clip(g_prime_inverse(-dVk/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*k*g(xstar) - dVk, l_param), 0, 1)
        return hstar*k*xstar - delta*k

Begin iterations

In [52]:
V = copy.copy(v_terminal)
v[:, :, T-1] = copy.copy(V)

In [69]:
# Construct forward difference for a and k
dVaF[0:na-1, :] = (V[1:na, :] - V[0:na-1, :])/da
dVaF[na-1, :] = dVaF[na-2, :]

dVkF[:, 0:nk-1] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
dVkF[:, nk-1] = dVkF[:, nk-2]

dVaB[1:na, :] = (V[1:na, :] - V[0:na-1, :])/da
dVaB[0, :] = dVaB[1, :]

dVkB[:, 1:nk] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
dVkB[:, 0] = dVkB[:, 1]

# Verify value function is increasing in a
if np.sum(dVaF < 0) > 0: # Print message if there exists an element of dVaF < 0
    print('V is not monotonically increasing in a')

if np.sum(dVkF < 0) > 0: # Print message if there exists an element of dVkF < 0
    print('V is not monotonically increasing in k')

# Compute consumption, which depends only on dVa via FOCs
cF = u_prime_inverse(dVaF, c_param)
cB = u_prime_inverse(dVaB, c_param)

# Copmpute x, which depends only on dVa and dVk (if h > 0). Corners at 0 and 1 taken into account.
xFF = np.clip(g_prime_inverse(-dVkF/dVaF), 0, 1)
xFB = np.clip(g_prime_inverse(-dVkB/dVaF), 0, 1)
xBF = np.clip(g_prime_inverse(-dVkF/dVaB), 0, 1)
xBB = np.clip(g_prime_inverse(-dVkB/dVaB), 0, 1)

# FOC for h, given x. Corners at 0 and 1 taken into account.
hFF = np.clip(uh_prime_inverse(-dVaF*kk*g(xFF) - dVkF, l_param), 0, 1)
hFB = np.clip(uh_prime_inverse(-dVaF*kk*g(xFB) - dVkB, l_param), 0, 1)
hBF = np.clip(uh_prime_inverse(-dVaB*kk*g(xBF) - dVkF, l_param), 0, 1)
hBB = np.clip(uh_prime_inverse(-dVaB*kk*g(xBB) - dVkB, l_param), 0, 1)

# Drift matrices for a
mu_aFF = hFF*kk*g(xFF) + r*aa - cF
mu_aFB = hFB*kk*g(xFB) + r*aa - cF
mu_aBF = hBF*kk*g(xBF) + r*aa - cB
mu_aBB = hBB*kk*g(xBB) + r*aa - cB

# Drift matrices for k
mu_kFF = hFF*kk*xFF - delta*kk
mu_kFB = hFB*kk*xFB - delta*kk
mu_kBF = hBF*kk*xBF - delta*kk
mu_kBB = hBB*kk*xBB - delta*kk

# Create indicator matrices indicating consistency of direction of drift with fwd/bwd difference
# Impose False at boundary if fwd/bwd difference implies drift sends state beyond boundary
I_FF = np.logical_and(mu_aFF > 0, mu_kFF > 0)
I_FF[na-1, :] = False
I_FF[:, nk-1] = False

I_FB = np.logical_and(mu_aFB > 0, mu_kFB < 0)
I_FB[na-1, :] = False
I_FB[:, 0] = False

I_BF = np.logical_and(mu_aBF < 0, mu_kFF > 0)
I_BF[0, :] = False
I_BF[:, nk-1] = False

I_BB = np.logical_and(mu_aBB < 0, mu_kFB < 0)
I_BB[0, :] = False
I_BB[:, 0] = False

# Compute validity of fwd/bwd difference combinations
# Stack all indicator matrices
I_stacked = np.zeros((na, nk, 4))
I_stacked[:, :, 0] = I_FF
I_stacked[:, :, 1] = I_FB
I_stacked[:, :, 2] = I_BF
I_stacked[:, :, 3] = I_BB

# Use logical_or over all 4 combinations; if at least one is valid, I_valid is True
I_valid = np.logical_or.reduce((I_FF, I_FB, I_BF, I_BB))
I_type = np.argmax(I_stacked, axis = 2)

# Impose state boundary condition for a_min
# First look at the case with mu_kBF
# Loop over a_min (upper edge) in the j dimension
aind = 0
for j in range(nk):
    if I_valid[aind, j] == False:
        dVkj = dVkF[aind, j]
        aval = aa[aind, 0]
        kj = kk[aind, j]

        # Find dVaB that satisfies FOC and a_min boundary conditio
        dVa_guess = dVaB[aind, j]
        dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
        xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
        cstar = u_prime_inverse(dVaj, c_param)

        mu_aj = hstar*kj*g(xstar) + r*aval - cstar
        mu_kj = hstar*kj*xstar - delta*kj

        if mu_kj >= 0 and j != nk - 1:
            # fwd diff for k is successful; BF scheme works
            I_valid[aind, j] = True
            I_type[aind, j] = 2

            # Replace bwd diff approximation for aval at j
            dVaB[aind, j] = dVaj

            # Replace bwd diff drift values
            mu_aBF[aind, j] = mu_aj
            mu_kBF[aind, j] = mu_kj

        # BF scheme failed; try BB scheme
        else:
            dVkj = dVkB[aind, j]
            dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

            xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
            cstar = u_prime_inverse(dVaj, c_param)

            # Evaluate if drift is consistent with chosen bwd diff for k
            mu_aj = hstar*kj*g(xstar) + r*aval - cstar
            mu_kj = hstar*kj*xstar - delta*kj


            if mu_kj <= 0 and j != 0:
                # bwd diff for k is successful; BB scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 3

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aBB[aind, j] = mu_aj
                mu_kBB[aind, j] = mu_kj

# Impose state boundary condition for a_max
# First look at the case with mu_kBF
# Loop over a_max (upper edge) in the j dimension
aind = na-1
for j in range(nk):
    if I_valid[aind, j] == False:
        dVkj = dVkF[aind, j]
        aval = aa[aind, 0]
        kj = kk[aind, j]

        # Find dVaF that satisfies FOC and a_max boundary condition
        dVa_guess = dVaF[aind, j]
        dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
        xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
        cstar = u_prime_inverse(dVaj, c_param)

        mu_aj = hstar*kj*g(xstar) + r*aval - cstar
        mu_kj = hstar*kj*xstar - delta*kj

        if mu_kj >= 0 and j != nk - 1:
            # fwd diff for k is successful; FF scheme works
            I_valid[aind, j] = True
            I_type[aind, j] = 0

            # Replace fwd diff approximation for aval at j
            dVaF[aind, j] = dVaj

            # Replace bwd diff drift values
            mu_aFF[aind, j] = mu_aj
            mu_kFF[aind, j] = mu_kj

        # BF scheme failed; try BB scheme
        else:
            dVkj = dVkB[aind, j]
            dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

            xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
            cstar = u_prime_inverse(dVaj, c_param)

            # Evaluate if drift is consistent with chosen bwd diff for k
            mu_aj = hstar*kj*g(xstar) + r*aval - cstar
            mu_kj = hstar*kj*xstar - delta*kj

            if mu_kj <= 0 and j != 0:
                # bwd diff for k is successful; BB scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 1

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aFB[aind, j] = mu_aj
                mu_kFB[aind, j] = mu_kj

# Impose state boundary condition for k_min
# First look at the case with mu_aFB
# Loop over k_min (upper edge) in the i dimension
kind = 0
for i in range(na):
    if I_valid[i, kind] == False:
        dVai = dVaF[i, kind]
        kval = kk[0, kind]
        ai = aa[i, kind]

        # Find dVkB that satisfies FOC and k_min boundary condition
        dVk_guess = 0
        while True:
            if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                break
            else:
                dVk_guess += 1

        dVki = opt.brentq(boundary_dVk, 0, dVk_guess, args = (dVai, ai, kval, l_param))

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
        xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
        cstar = u_prime_inverse(dVai, c_param)

        mu_ai = hstar*kval*g(xstar) + r*ai - cstar
        mu_ki = hstar*kval*xstar - delta*kval

        if mu_ai >= 0 and i != na:
            # fwd diff for a is successful; FB scheme works
            I_valid[i, kind] = True
            I_type[i, kind] = 1

            # Replace bwd diff approximation for kmin at i
            dVkB[i, kind] = dVki

            # Replace bwd diff drift values
            mu_aFB[i, kind] = mu_ai
            mu_kFB[i, kind] = mu_ki

        # FB scheme failed, try BB
        else:
            dVai = dVaB[i, kind]

            # Find dVkB that satisfies FOC and k_min boundary condition
            dVk_guess = 0
            while True:
                if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                    break
                else:
                    dVk_guess += 1

            dVki = opt.brentq(boundary_dVk, 0, dVk_guess, args = (dVai, ai, kval, l_param))

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
            cstar = u_prime_inverse(dVai, c_param)

            mu_ai = hstar*kval*g(xstar) + r*ai - cstar
            mu_ki = hstar*kval*xstar - delta*kval

            if mu_ai <= 0 and i != 0:
                # bwd diff for a is successful; BB scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 3

                # Replace bwd diff approximation for amax at j
                dVkB[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aBB[i, kind] = mu_ai
                mu_kBB[i, kind] = mu_ki

# Impose state boundary condition for k_max
# First look at the case with mu_aFF
# Loop over k_max (upper edge) in the i dimension
kind = nk-1
for i in range(na):
    if I_valid[i, kind] == False:
        dVai = dVaF[i, kind]
        kval = kk[0, kind]
        ai = aa[i, kind]

        # Find dVkF that satisfies FOC and k_max boundary condition
        dVk_guess = 0
        while True:
            if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                break
            else:
                dVk_guess += 1

        dVki = opt.brentq(boundary_dVk, 0, dVk_guess, args = (dVai, ai, kval, l_param))

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
        xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
        cstar = u_prime_inverse(dVai, c_param)

        mu_ai = hstar*kval*g(xstar) + r*ai - cstar
        mu_ki = hstar*kval*xstar - delta*kval

        if mu_ai >= 0 and i != na:
            # fwd diff for a is successful; FF scheme works
            I_valid[i, kind] = True
            I_type[i, kind] = 0

            # Replace fwd diff approximation for kmax at i
            dVkF[i, kind] = dVki

            # Replace drift values
            mu_aFF[i, kind] = mu_ai
            mu_kFF[i, kind] = mu_ki

        # FF scheme failed, try BF
        else:
            dVai = dVaB[i, kind]

            # Find dVkF that satisfies FOC and k_max boundary condition
            dVk_guess = 0
            while True:
                if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                    break
                else:
                    dVk_guess += 1

            dVki = opt.brentq(boundary_dVk, 0, dVk_guess, args = (dVai, ai, kval, l_param))

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
            cstar = u_prime_inverse(dVai, c_param)

            mu_ai = hstar*kval*g(xstar) + r*ai - cstar
            mu_ki = hstar*kval*xstar - delta*kval

            if mu_ai <= 0 and i != 0:
                # bwd diff for a is successful; BF scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 2

                # Replace fwd diff approximation for kmax at i
                dVkF[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aBF[i, kind] = mu_ai
                mu_kBF[i, kind] = mu_ki

# Construct dVa and dVk matrices
# If no scheme is valid, use steady-state values, i.e. stay put
dVa = (dVaF*(I_type < 2) + dVaB*(I_type > 1))*I_valid + dVa0*(~I_valid) # FF & FB has I_type < 2, BF and BB has I_type > 1
dVk = (dVkF*(1 - np.mod(I_type, 2)) + dVkB*(np.mod(I_type, 2)))*I_valid + dVk0*(~I_valid) # FF & BF has even I_type

# Construct drift values
mu_a = (mu_aFF*(I_type == 0) + mu_aFB*(I_type == 1) + mu_aBF*(I_type == 2) + mu_aBB*(I_type == 3))*I_valid
mu_k = (mu_kFF*(I_type == 0) + mu_kFB*(I_type == 1) + mu_kBF*(I_type == 2) + mu_kBB*(I_type == 3))*I_valid

# Construct policy functions and utility
C = u_prime_inverse(dVa, c_param)
X = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
H = np.clip(uh_prime_inverse(-dVa*kk*g(X) - dVk, l_param), 0, 1)
U = u(C, c_param) + uh(H, l_param)

# Construct sparse diagonal matrix A
# Reshape mu_a using column-major (Fortran) order
mu_a_diag = np.reshape(mu_a, na*nk, order='F')/da

# -1 diagonal is mu_a<0, +1 diag is mu>0
# Shift elements forward 1 index as placement in upper diagonal in spdiags lops off the first element. Want diagonal to start with the 1st element.
mu_a_diag_pos = np.roll(np.clip(mu_a_diag, 0, None), 1)

# Shift elements back 1 index as placement in lower diagonal in spdiags lops off the last element. Want diagonal to start with the 2nd element.
mu_a_diag_neg = np.roll(-np.clip(mu_a_diag, None, 0), -1) 

# Reshape mu_k using column-major (Fortran) order
mu_k_diag = np.reshape(mu_k, na*nk, order='F')/dk

# -na diag is mu_k < 0, +na diag is mu_k > 0
# Shift elements forward na index as placement in upper diagonal in spdiags lops off the first na elements. Want diagonal to start with 1st element.
mu_k_diag_pos = np.roll(np.clip(mu_k_diag, 0, None), na)

# Shift elements back na index as placement in lower diagonal in spdiags lops off the last na elements. Want diagonal to start with the na'th element
mu_k_diag_neg = np.roll(-np.clip(mu_k_diag, None, 0), -na)

# Construct main diagonal
main_diag = np.reshape(-np.abs(mu_a)/da - np.abs(mu_k)/dk, na*nk, order='F')

A = spdiags(np.array([mu_k_diag_neg, mu_a_diag_neg, main_diag, mu_a_diag_pos, mu_k_diag_pos]), np.array([-na, -1, 0, 1, na]), na*nk, na*nk)

# Solve set of linear equations to obtain V for current period from V of next period
b = np.reshape(U + V/dt, na*nk, order='F')
B = (1/dt + rho)*np.ones((na*nk, na*nk)) - A
V_sol = np.linalg.solve(B, b)[0]

LinAlgError: Singular matrix

In [54]:
A_filled = A.toarray()

In [66]:
np.sum(A_filled, axis = 1).max()

1.7763568394002505e-15

_, inds = sympy.Matrix(A_filled).rref()

for ind in range(400):
    if ind not in inds:
        print(ind)

A_filled[:, 398]

# Next iteration ++++++++++++++++++++++++++++++++++++++++++++++++++++

In [16]:
V = copy.copy(np.reshape(V_sol, (na, nk), order='F'))
v[:, :, T-2] = copy.copy(V)

NameError: name 'V_sol' is not defined

In [17]:
# Construct forward difference for a and k
dVaF[0:na-1, :] = (V[1:na, :] - V[0:na-1, :])/da
dVaF[na-1, :] = dVaF[na-2, :]

dVkF[:, 0:nk-1] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
dVkF[:, nk-1] = dVkF[:, nk-2]

dVaB[1:na, :] = (V[1:na, :] - V[0:na-1, :])/da
dVaB[0, :] = dVaB[1, :]

dVkB[:, 1:nk] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
dVkB[:, 0] = dVkB[:, 1]

# Verify value function is increasing in a
if np.sum(dVaF < 0) > 0: # Print message if there exists an element of dVaF < 0
    print('V is not monotonically increasing in a')

if np.sum(dVkF < 0) > 0: # Print message if there exists an element of dVkF < 0
    print('V is not monotonically increasing in k')

# Compute consumption, which depends only on dVa via FOCs
cF = u_prime_inverse(dVaF, c_param)
cB = u_prime_inverse(dVaB, c_param)

# Copmpute x, which depends only on dVa and dVk (if h > 0). Corners at 0 and 1 taken into account.
xFF = np.clip(g_prime_inverse(-dVkF/dVaF), 0, 1)
xFB = np.clip(g_prime_inverse(-dVkB/dVaF), 0, 1)
xBF = np.clip(g_prime_inverse(-dVkF/dVaB), 0, 1)
xBB = np.clip(g_prime_inverse(-dVkB/dVaB), 0, 1)

# FOC for h, given x. Corners at 0 and 1 taken into account.
hFF = np.clip(uh_prime_inverse(-dVaF*kk*g(xFF) - dVkF, l_param), 0, 1)
hFB = np.clip(uh_prime_inverse(-dVaF*kk*g(xFB) - dVkB, l_param), 0, 1)
hBF = np.clip(uh_prime_inverse(-dVaB*kk*g(xBF) - dVkF, l_param), 0, 1)
hBB = np.clip(uh_prime_inverse(-dVaB*kk*g(xBB) - dVkB, l_param), 0, 1)

# Drift matrices for a
mu_aFF = hFF*kk*g(xFF) + r*aa - cF
mu_aFB = hFB*kk*g(xFB) + r*aa - cF
mu_aBF = hBF*kk*g(xBF) + r*aa - cB
mu_aBB = hBB*kk*g(xBB) + r*aa - cB

# Drift matrices for k
mu_kFF = hFF*kk*xFF - delta*kk
mu_kFB = hFB*kk*xFB - delta*kk
mu_kBF = hBF*kk*xBF - delta*kk
mu_kBB = hBB*kk*xBB - delta*kk

# Create indicator matrices indicating consistency of direction of drift with fwd/bwd difference
# Impose False at boundary if fwd/bwd difference implies drift sends state beyond boundary
I_FF = np.logical_and(mu_aFF > 0, mu_kFF > 0)
I_FF[na-1, :] = False
I_FF[:, nk-1] = False

I_FB = np.logical_and(mu_aFB > 0, mu_kFB < 0)
I_FB[na-1, :] = False
I_FB[:, 0] = False

I_BF = np.logical_and(mu_aBF < 0, mu_kFF > 0)
I_BF[0, :] = False
I_BF[:, nk-1] = False

I_BB = np.logical_and(mu_aBB < 0, mu_kFB < 0)
I_BB[0, :] = False
I_BB[:, 0] = False

# Compute validity of fwd/bwd difference combinations
# Stack all indicator matrices
I_stacked = np.zeros((na, nk, 4))
I_stacked[:, :, 0] = I_FF
I_stacked[:, :, 1] = I_FB
I_stacked[:, :, 2] = I_BF
I_stacked[:, :, 3] = I_BB

# Use logical_or over all 4 combinations; if at least one is valid, I_valid is True
I_valid = np.logical_or.reduce((I_FF, I_FB, I_BF, I_BB))
I_type = np.argmax(I_stacked, axis = 2)

# Impose state boundary condition for a_min
# First look at the case with mu_kBF
# Loop over a_min (upper edge) in the j dimension
aind = 0
for j in range(nk):
    if I_valid[aind, j] == False:
        dVkj = dVkF[aind, j]
        aval = aa[aind, 0]
        kj = kk[aind, j]

        # Find dVaB that satisfies FOC and a_min boundary conditio
        dVa_guess = dVaB[aind, j]
        dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
        xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
        cstar = u_prime_inverse(dVaj, c_param)

        mu_aj = hstar*kj*g(xstar) + r*aval - cstar
        mu_kj = hstar*kj*xstar - delta*kj

        if mu_kj >= 0 and j != nk - 1:
            # fwd diff for k is successful; BF scheme works
            I_valid[aind, j] = True
            I_type[aind, j] = 2

            # Replace bwd diff approximation for aval at j
            dVaB[aind, j] = dVaj

            # Replace bwd diff drift values
            mu_aBF[aind, j] = mu_aj
            mu_kBF[aind, j] = mu_kj

        # BF scheme failed; try BB scheme
        else:
            dVkj = dVkB[aind, j]
            dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

            xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
            cstar = u_prime_inverse(dVaj, c_param)

            # Evaluate if drift is consistent with chosen bwd diff for k
            mu_aj = hstar*kj*g(xstar) + r*aval - cstar
            mu_kj = hstar*kj*xstar - delta*kj


            if mu_kj <= 0 and j != 0:
                # bwd diff for k is successful; BB scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 3

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aBB[aind, j] = mu_aj
                mu_kBB[aind, j] = mu_kj

# Impose state boundary condition for a_max
# First look at the case with mu_kBF
# Loop over a_max (upper edge) in the j dimension
aind = na-1
for j in range(nk):
    if I_valid[aind, j] == False:
        dVkj = dVkF[aind, j]
        aval = aa[aind, 0]
        kj = kk[aind, j]

        # Find dVaF that satisfies FOC and a_max boundary condition
        dVa_guess = dVaF[aind, j]
        dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
        xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
        cstar = u_prime_inverse(dVaj, c_param)

        mu_aj = hstar*kj*g(xstar) + r*aval - cstar
        mu_kj = hstar*kj*xstar - delta*kj

        if mu_kj >= 0 and j != nk - 1:
            # fwd diff for k is successful; FF scheme works
            I_valid[aind, j] = True
            I_type[aind, j] = 0

            # Replace fwd diff approximation for aval at j
            dVaF[aind, j] = dVaj

            # Replace bwd diff drift values
            mu_aFF[aind, j] = mu_aj
            mu_kFF[aind, j] = mu_kj

        # BF scheme failed; try BB scheme
        else:
            dVkj = dVkB[aind, j]
            dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

            xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
            cstar = u_prime_inverse(dVaj, c_param)

            # Evaluate if drift is consistent with chosen bwd diff for k
            mu_aj = hstar*kj*g(xstar) + r*aval - cstar
            mu_kj = hstar*kj*xstar - delta*kj

            if mu_kj <= 0 and j != 0:
                # bwd diff for k is successful; BB scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 1

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aFB[aind, j] = mu_aj
                mu_kFB[aind, j] = mu_kj

# Impose state boundary condition for k_min
# First look at the case with mu_aFB
# Loop over k_min (upper edge) in the i dimension
kind = 0
for i in range(na):
    if I_valid[i, kind] == False:
        dVai = dVaF[i, kind]
        kval = kk[0, kind]
        ai = aa[i, kind]

        # Find dVkB that satisfies FOC and k_min boundary condition
        dVk_guess = 0
        while True:
            if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                break
            else:
                dVk_guess += 1

        dVki = opt.brentq(boundary_dVk, 0, dVk_guess, args = (dVai, ai, kval, l_param))

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
        xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
        cstar = u_prime_inverse(dVai, c_param)

        mu_ai = hstar*kval*g(xstar) + r*ai - cstar
        mu_ki = hstar*kval*xstar - delta*kval

        if mu_ai >= 0 and i != na:
            # fwd diff for a is successful; FB scheme works
            I_valid[i, kind] = True
            I_type[i, kind] = 1

            # Replace bwd diff approximation for kmin at i
            dVkB[i, kind] = dVki

            # Replace bwd diff drift values
            mu_aFB[i, kind] = mu_ai
            mu_kFB[i, kind] = mu_ki

        # FB scheme failed, try BB
        else:
            dVai = dVaB[i, kind]

            # Find dVkB that satisfies FOC and k_min boundary condition
            dVk_guess = 0
            while True:
                if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                    break
                else:
                    dVk_guess += 1

            dVki = opt.brentq(boundary_dVk, 0, dVk_guess, args = (dVai, ai, kval, l_param))

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
            cstar = u_prime_inverse(dVai, c_param)

            mu_ai = hstar*kval*g(xstar) + r*ai - cstar
            mu_ki = hstar*kval*xstar - delta*kval

            if mu_ai <= 0 and i != 0:
                # bwd diff for a is successful; BB scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 3

                # Replace bwd diff approximation for amax at j
                dVkB[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aBB[i, kind] = mu_ai
                mu_kBB[i, kind] = mu_ki

# Impose state boundary condition for k_max
# First look at the case with mu_aFF
# Loop over k_max (upper edge) in the i dimension
kind = nk-1
for i in range(na):
    if I_valid[i, kind] == False:
        dVai = dVaF[i, kind]
        kval = kk[0, kind]
        ai = aa[i, kind]

        # Find dVkF that satisfies FOC and k_max boundary condition
        dVk_guess = 0
        while True:
            if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                break
            else:
                dVk_guess += 1

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
        xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
        cstar = u_prime_inverse(dVai, c_param)

        mu_ai = hstar*kval*g(xstar) + r*ai - cstar
        mu_ki = hstar*kval*xstar - delta*kval

        if mu_ai >= 0 and i != na:
            # fwd diff for a is successful; FF scheme works
            I_valid[i, kind] = True
            I_type[i, kind] = 0

            # Replace fwd diff approximation for kmax at i
            dVkF[i, kind] = dVki

            # Replace drift values
            mu_aFF[i, kind] = mu_ai
            mu_kFF[i, kind] = mu_ki

        # FF scheme failed, try BF
        else:
            dVai = dVaB[i, kind]

            # Find dVkF that satisfies FOC and k_max boundary condition
            dVk_guess = 0
            while True:
                if boundary_dVk(dVk_guess, dVai, ai, kval, l_param) > 0:
                    break
                else:
                    dVk_guess += 1

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
            cstar = u_prime_inverse(dVai, c_param)

            mu_ai = hstar*kval*g(xstar) + r*ai - cstar
            mu_ki = hstar*kval*xstar - delta*kval

            if mu_ai <= 0 and i != 0:
                # bwd diff for a is successful; BF scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 2

                # Replace fwd diff approximation for kmax at i
                dVkF[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aBF[i, kind] = mu_ai
                mu_kBF[i, kind] = mu_ki

# Construct dVa and dVk matrices
# If no scheme is valid, use steady-state values, i.e. stay put
dVa = (dVaF*(I_type < 2) + dVaB*(I_type > 1))*I_valid + dVa0*(~I_valid) # FF & FB has I_type < 2, BF and BB has I_type > 1
dVk = (dVkF*(1 - np.mod(I_type, 2)) + dVkB*(np.mod(I_type, 2)))*I_valid + dVk0*(~I_valid) # FF & BF has even I_type

# Construct drift values
mu_a = (mu_aFF*(I_type == 0) + mu_aFB*(I_type == 1) + mu_aBF*(I_type == 2) + mu_aBB*(I_type == 3))*I_valid
mu_k = (mu_kFF*(I_type == 0) + mu_kFB*(I_type == 1) + mu_kBF*(I_type == 2) + mu_kBB*(I_type == 3))*I_valid

# Construct policy functions and utility
C = u_prime_inverse(dVa, c_param)
X = np.clip(g_prime_inverse(-dVk/dVa), 0, 1)
H = np.clip(uh_prime_inverse(-dVa*kk*g(X) - dVk, l_param), 0, 1)
U = u(C, c_param) + uh(H, l_param)

# Construct sparse diagonal matrix A
# Reshape mu_a using column-major (Fortran) order
mu_a_diag = np.reshape(mu_a, na*nk, order='F')/da

# -1 diagonal is mu_a<0, +1 diag is mu>0
# Shift elements forward 1 index as placement in upper diagonal in spdiags lops off the first element. Want diagonal to start with the 1st element.
mu_a_diag_pos = np.roll(np.clip(mu_a_diag, 0, None), 1)

# Shift elements back 1 index as placement in lower diagonal in spdiags lops off the last element. Want diagonal to start with the 2nd element.
mu_a_diag_neg = np.roll(-np.clip(mu_a_diag, None, 0), -1) 

# Reshape mu_k using column-major (Fortran) order
mu_k_diag = np.reshape(mu_k, na*nk, order='F')/dk

# -na diag is mu_k < 0, +na diag is mu_k > 0
# Shift elements forward na index as placement in upper diagonal in spdiags lops off the first na elements. Want diagonal to start with 1st element.
mu_k_diag_pos = np.roll(np.clip(mu_k_diag, 0, None), na)

# Shift elements back na index as placement in lower diagonal in spdiags lops off the last na elements. Want diagonal to start with the na'th element
mu_k_diag_neg = np.roll(-np.clip(mu_k_diag, None, 0), -na)

# Construct main diagonal
main_diag = np.reshape(-np.abs(mu_a)/da - np.abs(mu_k)/dk, na*nk, order='F')

A = spdiags(np.array([mu_k_diag_neg, mu_a_diag_neg, main_diag, mu_a_diag_pos, mu_k_diag_pos]), np.array([-na, -1, 0, 1, na]), na*nk, na*nk)

# Solve set of linear equations to obtain V for current period from V of next period
b = np.reshape(U + V/dt, na*nk, order='F')
B = (1/dt + rho)*np.ones(na*nk) - A
V_sol = np.linalg.solve(B, b)

LinAlgError: Singular matrix